In [ ]:
#date time needs fixing
# remove userref
# remove hastags - keep works (iterable)
# urls
# unescaped html chars

In [ ]:
import pandas as pd
import seaborn as sns

from pyspark.sql import SparkSession as ss, functions as f
spark = ss.builder.getOrCreate()

pd.options.display.max_columns = None
sns.set(color_codes=True)
timestampFormat="EEE MMM dd HH:mm:ss z yyyy"
IN_PATH = "RAW"
PATH2= "RAW"
OUT_PATH ="CLEAN"

schema="polarity FLOAT, id LONG, date_time TIMESTAMP, query string, user STRING, text STRING"
spark_reader = spark.read.schema(schema)

In [69]:
import html
@f.udf

def html_unescape(s):
    if(s):
        return html.unescape(s)
    else:
        return s
    
def clean_data(df):
    df_clean = (
    df
    .withColumn("text", f.regexp_replace(f.col("text"), url_regex, ""))
    .withColumn("text", f.regexp_replace(f.col("text"), email_regex, ""))
    .withColumn("text", f.regexp_replace(f.col("text"), user_regex, ""))
    .withColumn("text", f.regexp_replace(f.col("text"), "#", " "))
    .withColumn("text", html_unescape(f.col("text")))
    .filter(f.col("text") != '')
    )
    return df_clean
    
raw_df = spark_reader.csv(IN_PATH, timestampFormat=timestampFormat).cache()
clean_df = clean_data(raw_df)

clean_df.write.partitionBy("polarity").parquet(OUT_PATH, mode="overwrite")

?Obama Administration Must Stop Bonuses to AIG Ponzi Schemers ... http://bit.ly/2CUIg
@spinuzzi: Has been a bit crazy, with steep learning curve, but LyX is really good for long docs. For anything shorter, it would be insane.
@phyreman9 Google is always a good place to look. Should've mentioned I worked on the Mustang w/ my Dad, @KimbleT.
[#MLUC09] Customer Innovation Award Winner: Booz Allen Hamilton -- http://ping.fm/c2hPP
#lebron best athlete of our generation, if not all time (basketball related) I don't want to get into inter-sport debates about   __1/2
localhost:1234
some@email.adress
some.email@domain.com
a_much_more_complicated_email.some.email@domain.with.lots.of.dots.and_underscores.longdomainname
www.something.com
http://spark.apache.org/
file://some_file_location/file.fmt
file:///some_file_location/file.fmt
https://blip.fm/~6emhv
https://www.freeformatter.com/java-regex-tester.html#ad-output
https://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.evaluation
google@gmail.com

In [70]:
user_regex = r"(@\w{1,15})"
hash_regex = r"(#\w{1})"
url_regex = r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)"
email_regex = r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"

In [71]:
clean_df.count()

1597303